## Data Analysis - empathetic dialogues (ParlAI - Facebook Research)

###### **Original Dataset:** [HuggingFace](https://huggingface.co/datasets/empathetic_dialogues) -- [Facebook Research (GitHub)](https://github.com/facebookresearch/EmpatheticDialogues) -- [ParlAI](https://parl.ai/docs/tasks.html#empathetic-dialogues)
###### **Students:** Yacine MOKHTARI & Lilia IZRI

### Lectures des données + rapide analyse

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

PATH = "./"
FILENAMES = ["test", "train", "valid"]
COLNAMES = ["conv_id", "utterance_idx", "context", "prompt", "utterance"]
df_test = pd.read_csv(PATH+FILENAMES[0]+".csv", usecols=COLNAMES)
df_train = pd.read_csv(PATH+FILENAMES[1]+".csv", usecols=COLNAMES)
df_valid = pd.read_csv(PATH+FILENAMES[2]+".csv", usecols=COLNAMES)

In [2]:
# Un aperçu :
df_train.head(10)

,conv_id,utterance_idx,context,prompt,utterance
0,hit:0_conv:1,1,sentimental,I remember going to the fireworks with my best...,I remember going to see the fireworks with my ...
1,hit:0_conv:1,2,sentimental,I remember going to the fireworks with my best...,Was this a friend you were in love with_comma_...
2,hit:0_conv:1,3,sentimental,I remember going to the fireworks with my best...,This was a best friend. I miss her.
3,hit:0_conv:1,4,sentimental,I remember going to the fireworks with my best...,Where has she gone?
4,hit:0_conv:1,5,sentimental,I remember going to the fireworks with my best...,We no longer talk.
5,hit:0_conv:1,6,sentimental,I remember going to the fireworks with my best...,Oh was this something that happened because of...
6,hit:1_conv:2,1,afraid,i used to scare for darkness,it feels like hitting to blank wall when i se...
7,hit:1_conv:2,2,afraid,i used to scare for darkness,Oh ya? I don't really see how
8,hit:1_conv:2,3,afraid,i used to scare for darkness,dont you feel so.. its a wonder
9,hit:1_conv:2,4,afraid,i used to scare for darkness,I do actually hit blank walls a lot of times b...


In [3]:
df_test.isna().sum(), df_train.isna().sum(), df_valid.isna().sum()

(conv_id          0
 utterance_idx    0
 context          0
 prompt           0
 utterance        0
 dtype: int64,
 conv_id          0
 utterance_idx    0
 context          0
 prompt           0
 utterance        0
 dtype: int64,
 conv_id          0
 utterance_idx    0
 context          0
 prompt           0
 utterance        0
 dtype: int64)

In [4]:
print("Test length :", len(df_test))
print("Train length :", len(df_train))
print("valid length :", len(df_valid))
print("")
print("Nombre total de données :", len(df_test)+len(df_train)+len(df_valid))

Test length : 10943
Train length : 76673
valid length : 12030

Nombre total de données : 99646


In [5]:
print("Test :", df_test.duplicated().sum(), " dupliqué(s) !")
print("Train :", df_train.duplicated().sum(), " dupliqué(s) !")
print("Valid :", df_valid.duplicated().sum(), " dupliqué(s) !")

Test : 0  dupliqué(s) !
Train : 0  dupliqué(s) !
Valid : 0  dupliqué(s) !


In [6]:
# retirer les dupliquées
df_test = df_test[~df_test.duplicated()]
df_train = df_train[~df_train.duplicated()]
df_valid = df_valid[~df_valid.duplicated()]

print("Test length :", len(df_test))
print("Train length :", len(df_train))
print("valid length :", len(df_valid))
print("")
print("Nombre total de données après suppression de dupliqués:", len(df_test)+len(df_train)+len(df_valid))

Test length : 10943
Train length : 76673
valid length : 12030

Nombre total de données après suppression de dupliqués: 99646


* Au départ j'ai pas pris les deux premières colonnes, so j'avais des dupliquées mais je comprend mieux mtn :) 

In [7]:
### Regarder si l'intersection est vide 
if ((len(pd.merge(df_test, df_train, how="inner")), len(pd.merge(df_test, df_valid, how="inner")), len(pd.merge(df_valid, df_train, how="inner"))) == (0,0,0)):
    print("Il n y a pas d'intersection entre les ensemble !")
else:
    print("Intersection détectée, regarde en détail :)")

Il n y a pas d'intersection entre les ensemble !


### Reconstitution du dataframe global
1. Union des trois df
2. Comprendre mieux le df
3. Trier et grouper pour reproduire les scénarios/dialogues 

In [8]:
# Union
frames = [df_test, df_train, df_valid]
df = pd.concat(frames)
df.head(10)

,conv_id,utterance_idx,context,prompt,utterance
0,hit:0_conv:0,1,guilty,I felt guilty when I was driving home one nigh...,Yeah about 10 years ago I had a horrifying exp...
1,hit:0_conv:0,2,guilty,I felt guilty when I was driving home one nigh...,Did you suffer any injuries?
2,hit:0_conv:0,3,guilty,I felt guilty when I was driving home one nigh...,No I wasn't hit. It turned out they were drunk...
3,hit:0_conv:0,4,guilty,I felt guilty when I was driving home one nigh...,Why did you feel guilty? People really shouldn...
4,hit:0_conv:0,5,guilty,I felt guilty when I was driving home one nigh...,I don't know I was new to driving and hadn't e...
5,hit:34_conv:69,1,caring,My mother stopped by my house one day and said...,Well_comma_ can you tell me about your experie...
6,hit:34_conv:69,2,caring,My mother stopped by my house one day and said...,Yeah i wanted to tell you about the time i was...
7,hit:34_conv:69,3,caring,My mother stopped by my house one day and said...,Oh my goodness_comma_ that's very scary! I hop...
8,hit:34_conv:69,4,caring,My mother stopped by my house one day and said...,Yeah he was punished hes in jail still
9,hit:37_conv:74,1,lonely,I just broke up with my girlfriend_comma_ we w...,I there_comma_ dont know what to do_comma_ jst...


In [9]:
# Comprendre qu'il existe plusieurs "dialogues/scénarios" pour un même contexte
# et que ce qui les diffèrent est le "prompt"
df[df.context.values == "guilty"].head(10)

,conv_id,utterance_idx,context,prompt,utterance
0,hit:0_conv:0,1,guilty,I felt guilty when I was driving home one nigh...,Yeah about 10 years ago I had a horrifying exp...
1,hit:0_conv:0,2,guilty,I felt guilty when I was driving home one nigh...,Did you suffer any injuries?
2,hit:0_conv:0,3,guilty,I felt guilty when I was driving home one nigh...,No I wasn't hit. It turned out they were drunk...
3,hit:0_conv:0,4,guilty,I felt guilty when I was driving home one nigh...,Why did you feel guilty? People really shouldn...
4,hit:0_conv:0,5,guilty,I felt guilty when I was driving home one nigh...,I don't know I was new to driving and hadn't e...
224,hit:291_conv:582,1,guilty,I waited at a restaurant for 30 minutes befor...,Did I tell you what happened with Li the other...
225,hit:291_conv:582,2,guilty,I waited at a restaurant for 30 minutes befor...,No_comma_ you didn't tell me! What happened?
226,hit:291_conv:582,3,guilty,I waited at a restaurant for 30 minutes befor...,She's 45 minutes late to our anniversary dinne...
227,hit:291_conv:582,4,guilty,I waited at a restaurant for 30 minutes befor...,Oh no! Is she okay?! Don't beat yourself up to...
260,hit:337_conv:674,1,guilty,I felt bad recently because I was dishonest wi...,I felt pretty bad when I was having girl talk ...


Nous remarquons ainsi que :
* Il existe plusieurs "dialogues/scénarios" pour un même contexte
* "utterence_idx" indique l'ordre des messages "utterence" pour un "conv_id" donnée

Ainsi, il conviendrait peut-être de trier le jeu de données suivant les colonnes ["conv_id", "utterance_idx"] et de les grouper selon les colonnes ["context", "prompt"].

In [10]:
df_sorted = df.sort_values(by=["conv_id", "utterance_idx"])
df_sorted.head(10)

,conv_id,utterance_idx,context,prompt,utterance
0,hit:0_conv:0,1,guilty,I felt guilty when I was driving home one nigh...,Yeah about 10 years ago I had a horrifying exp...
1,hit:0_conv:0,2,guilty,I felt guilty when I was driving home one nigh...,Did you suffer any injuries?
2,hit:0_conv:0,3,guilty,I felt guilty when I was driving home one nigh...,No I wasn't hit. It turned out they were drunk...
3,hit:0_conv:0,4,guilty,I felt guilty when I was driving home one nigh...,Why did you feel guilty? People really shouldn...
4,hit:0_conv:0,5,guilty,I felt guilty when I was driving home one nigh...,I don't know I was new to driving and hadn't e...
0,hit:0_conv:1,1,sentimental,I remember going to the fireworks with my best...,I remember going to see the fireworks with my ...
1,hit:0_conv:1,2,sentimental,I remember going to the fireworks with my best...,Was this a friend you were in love with_comma_...
2,hit:0_conv:1,3,sentimental,I remember going to the fireworks with my best...,This was a best friend. I miss her.
3,hit:0_conv:1,4,sentimental,I remember going to the fireworks with my best...,Where has she gone?
4,hit:0_conv:1,5,sentimental,I remember going to the fireworks with my best...,We no longer talk.


In [11]:
def extraireSelonDialogue(dataframe):
    question = []
    reponse =  []
    ctx = []

    qst = True # Si à vrai, ajouter en tant que question, si faux, ajouter en tant que réponse
    for i in tqdm(np.arange(len(dataframe))):
        tmp = dataframe.iloc[i]

        # un nouveau scénario/dialogue
        if (tmp["utterance_idx"] == 1):
            # si le dernier ajout était une question, 
            # NaN pour la réponse 
            if qst == False :
                reponse.append("")
                qst = True
        
        # Ajout de la Question / Réponse
        if qst:
                question.append(tmp.utterance.replace("_comma_", ","))
        else : 
            reponse.append(tmp.utterance.replace("_comma_", ","))
        qst = not qst

        # Ajout du contexte (émotion)
        ctx = tmp["context"]

    return question, reponse, ctx

In [12]:
q, r, c = extraireSelonDialogue(df_sorted)
assert len(q) == len(r)

100%|██████████| 99646/99646 [00:14<00:00, 6690.19it/s]


In [13]:
df_QA = pd.DataFrame(data =  { "Context" : c,  "Question" : q, "Reponse" : r})
df_QA.head(20)

,Context,Question,Reponse
0,embarrassed,Yeah about 10 years ago I had a horrifying exp...,Did you suffer any injuries?
1,embarrassed,No I wasn't hit. It turned out they were drunk...,Why did you feel guilty? People really shouldn...
2,embarrassed,I don't know I was new to driving and hadn't e...,
3,embarrassed,I remember going to see the fireworks with my ...,"Was this a friend you were in love with, or ju..."
4,embarrassed,This was a best friend. I miss her.,Where has she gone?
5,embarrassed,We no longer talk.,Oh was this something that happened because of...
6,embarrassed,My girlfriend got me a pet toad today!,Do you like toads?
7,embarrassed,I do! I was so happy when I opened the box and...,"That was nice of your girlfriend, do you love ..."
8,embarrassed,I really like the new paint job on my house.,That's nice. What color did you paint it?
9,embarrassed,I went with blue and yellow.,Those are swell colors! I am sure they will br...


In [14]:
# Enlever les questions sans réponses (fin de conv)
mask = (df_QA["Reponse"].values == "")
print("Il y a", mask.sum(), " questions sans réponses.")

Il y a 4468  questions sans réponses.


In [15]:
df_QA[mask].head(12)

,Context,Question,Reponse
2,embarrassed,I don't know I was new to driving and hadn't e...,
18,embarrassed,"No, he doesn't knows",
27,embarrassed,Statistics exam. I aced this class for sure. T...,
36,embarrassed,Thanks,
43,embarrassed,Yeah my family. We're going to the beach.,
50,embarrassed,I am taking to a movie and the Olive Garden.,
71,embarrassed,No not yet. He's sitting here playing Battlef...,
76,embarrassed,I got all of my housework done. For the rest o...,
83,embarrassed,I know but now I am afraid he will never return,
100,embarrassed,Wow. Looks like you are doing a good job.,


In [16]:
# Garder que les questions AVEC réponse 
# François Landes : "C'est toujours triste de supprimer des données" (oui bon, lui il dit "features" xD)
df_QA = df_QA[~mask]

#### Sauvegarder le jeu de données ainsi obtenu !

In [17]:
OUTPUT_PATH = "./output/"
OUTPUT_NAME = "EmpatheticDialogues.tsv"
OUTPUT_NAME2 = "EmpatheticDialogues_wo_context.tsv"

df_QA.to_csv(OUTPUT_PATH + OUTPUT_NAME, sep="\t")
df_QA[["Question", "Reponse"]].to_csv(OUTPUT_PATH + OUTPUT_NAME2, sep="\t")